# Setting Up Environment

In [1]:
%%capture

import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

from sklearn.preprocessing import StandardScaler

import gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv

In [2]:
# Joining predictions to table w/ results and getting result

predictions = pd.read_csv('mma_data_predictions.csv', index_col = 0)

data = pd.read_csv('mma_data.csv', index_col = 0)
data = data[data.result >= 0]
results_data = data[['fighter_1', 'fighter_2', 'result', 'KO_OVR', 'SUB_OVR']]

odds_data = pd.read_csv('mma_data_odds.csv', index_col = 0)

merged = predictions.merge(results_data, on = ['fighter_1', 'fighter_2'])

In [3]:
# Winner results
merged['Predicted_Result_RF'] = merged.Prediction_RF_Winner.apply(lambda x: 1 if x > 0.5 else 0)
merged['Predicted_Result_GB'] = merged.Prediction_GB_Winner.apply(lambda x: 1 if x > 0.5 else 0)
merged['Predicted_Result_LGBM'] = merged.Prediction_LGBM_Winner.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Predicted_Resulted_LR'] = merged.Prediction_LR_Winner.apply(lambda x: 1 if x > 0.5 else 0)
merged['Accurate_RF'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Result_RF else 0, axis = 1)
merged['Accurate_GB'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Result_GB else 0, axis = 1)
merged['Accurate_LGBM'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Result_LGBM else 0, axis = 1)

# # Sub results
# merged['Predicted_Sub_RF'] = merged.Prediction_RF_SUB.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Predicted_Sub_GB'] = merged.Prediction_GB_SUB.apply(lambda x: 1 if x > 0.5 else 0)
# # merged['Predicted_Resulted_LR'] = merged.Prediction_LR_Winner.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Accurate_RF_SUB'] = merged.apply(lambda x: 1 if x.SUB_OVR_y == x.Predicted_Sub_RF else 0, axis = 1)
# merged['Accurate_GB_SUB'] = merged.apply(lambda x: 1 if x.SUB_OVR_y == x.Predicted_Sub_GB else 0, axis = 1)

# # KO Results
# merged['Predicted_KO_RF'] = merged.Prediction_RF_KO.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Predicted_KO_GB'] = merged.Prediction_GB_KO.apply(lambda x: 1 if x > 0.5 else 0)
# # merged['Predicted_Resulted_LR'] = merged.Prediction_LR_Winner.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Accurate_RF_KO'] = merged.apply(lambda x: 1 if x.KO_OVR_y == x.Predicted_KO_RF else 0, axis = 1)
# merged['Accurate_GB_KO'] = merged.apply(lambda x: 1 if x.KO_OVR_y == x.Predicted_KO_GB else 0, axis = 1)

# Getting all the relevant data in one place for bet constructs
odds_data = odds_data[['fighter_1', 'fighter_2', 'Fighter_1_Odds', 'Fighter_2_Odds']]
profit_df = merged.merge(odds_data, on = ['fighter_1', 'fighter_2'])
profit_df = profit_df[(profit_df.Fighter_1_Odds!=0) & (profit_df.Fighter_2_Odds!=0)]

# Machine Learning

### Analyzing Accuracy

In [4]:
print(f'Winner accuracy for RF is: {merged.Accurate_RF.mean()*100}%')
print(f'Winner accuracy for GB is: {merged.Accurate_GB.mean()*100}%')
print(f'Winner accuracy for LGBM is: {merged.Accurate_LGBM.mean()*100}%')
# print(f'Sub accuracy for RF is: {merged.Accurate_RF_SUB.mean()*100}%')
# print(f'Sub accuracy for GB is: {merged.Accurate_GB_SUB.mean()*100}%')
# print(f'KO accuracy for RF is: {merged.Accurate_RF_KO.mean()*100}%')
# print(f'KO accuracy for GB is: {merged.Accurate_GB_KO.mean()*100}%')

Winner accuracy for RF is: 57.54189944134078%
Winner accuracy for GB is: 63.128491620111724%
Winner accuracy for LGBM is: 62.56983240223464%


##### Looking At Only Veteran Fights

In [5]:
merged['Fights_1'] = merged.wins_1 + merged.losses_1
merged['Fights_2'] = merged.wins_2 + merged.losses_2

test = merged[(merged.Fights_1 > 15) | (merged.Fights_2 > 15)]

In [6]:
print(f'Winner accuracy for RF is: {test.Accurate_RF.mean()*100}%')
print(f'Winner accuracy for veterans GB is: {test.Accurate_GB.mean()*100}%')
print(f'Winner accuracy for veterans LGBM is: {test.Accurate_LGBM.mean()*100}%')
# print(f'Sub accuracy for veterans RF is: {test.Accurate_RF_SUB.mean()*100}%')
# print(f'Sub accuracy for veterans GB is: {test.Accurate_GB_SUB.mean()*100}%')
# print(f'KO accuracy for veterans RF is: {test.Accurate_RF_KO.mean()*100}%')
# print(f'KO accuracy for veterans GB is: {test.Accurate_GB_KO.mean()*100}%')

Winner accuracy for RF is: 62.328767123287676%
Winner accuracy for veterans GB is: 65.75342465753424%
Winner accuracy for veterans LGBM is: 65.75342465753424%


# Potential Profit

### GB

In [7]:
# Straight

def calculate_payoff_and_result(row):
    # Calculating Payoff
    if row.Predicted_Result_GB == 1:
        if row.Fighter_1_Odds>0:
            payoff = (row.Fighter_1_Odds/100)*row.Bet
        else:
            payoff = row.Bet/((abs(row.Fighter_1_Odds)/100))
    else:
        if row.Fighter_2_Odds>0:
            payoff = (row.Fighter_2_Odds/100)*row.Bet
        else:
            payoff = row.Bet/((abs(row.Fighter_2_Odds)/100))
    # Calculating Bet Result
    if row.Predicted_Result_GB == row.result_y:
        bet_result = payoff
    else:
        bet_result = -(row.Bet)
    
    return bet_result
            
profit_df['Bet'] = 100
profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result, axis = 1)

profit_df.Bet_Result.sum()

20.821674187981273

In [8]:
# Veteran fights only

for num_fights in [10, 15, 20, 25]:

    profit_df['Fights_1'] = profit_df.wins_1 + profit_df.losses_1
    profit_df['Fights_2'] = profit_df.wins_2 + profit_df.losses_2

    test = profit_df[(profit_df.Fights_1 > num_fights) | (profit_df.Fights_2 > num_fights)]
    results = test.Bet_Result.sum()
    
    print(f'For a {num_fights} minimum, the model returns {results}')

For a 10 minimum, the model returns 95.49739062840345
For a 15 minimum, the model returns 537.6472849503133
For a 20 minimum, the model returns 316.86987734379954
For a 25 minimum, the model returns -410.1019616381073


In [9]:
# Betting with only a certain odds differential

def calculate_odds_internal(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))
    if odds>0:
        return (100/(odds+100))

def calculate_bets_internal(row, diff):
    bet = 0
    
    if row.Prediction_GB_Winner - calculate_odds_internal(row.Fighter_1_Odds) >= diff:
        bet = 100
    if (1.0 - row.Prediction_GB_Winner) - calculate_odds_internal(row.Fighter_2_Odds) >= diff:
        bet = 100
    
    return bet

def calculate_payoff_and_result_internal(row):

    if row.Bet > 0:
        # Calculating Payoff
        if row.Predicted_Result_GB == 1:
            if row.Fighter_1_Odds>0:
                payoff = (row.Fighter_1_Odds/100)*row.Bet
            else:
                payoff = row.Bet/((abs(row.Fighter_1_Odds)/100))
        else:
            if row.Fighter_2_Odds>0:
                payoff = (row.Fighter_2_Odds/100)*row.Bet
            else:
                payoff = row.Bet/((abs(row.Fighter_2_Odds)/100))
        # Calculating Bet Result
        if row.Predicted_Result_GB == row.result_y:
            bet_result = payoff
        else:
            bet_result = -(row.Bet)

    else:
        bet_result = 0

    return bet_result

best_diff = 0
best_profit = 0

for i in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
    
    profit_df['Bet'] = profit_df.apply(calculate_bets_internal, diff = i, axis = 1)
    profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result, axis = 1)

    print(f'With a cutoff of {i}, betting results are {profit_df.Bet_Result.sum()}')
    
    if float(profit_df.Bet_Result.sum()) > best_profit:
        best_diff = i
    
    if profit_df.Bet_Result.sum() > best_profit:
        best_profit = profit_df.Bet_Result.sum()

# Veteran fights only

profit_df['Bet'] = profit_df.apply(calculate_bets_internal, diff = best_diff, axis = 1)
profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result, axis = 1)

for num_fights in [10, 15, 20, 25]:

    profit_df['Fights_1'] = profit_df.wins_1 + profit_df.losses_1
    profit_df['Fights_2'] = profit_df.wins_2 + profit_df.losses_2

    test = profit_df[(profit_df.Fights_1 > num_fights) | (profit_df.Fights_2 > num_fights)]
    results = test.Bet_Result.sum()
    
    print(f'For a {num_fights} minimum, the model returns {results}')

With a cutoff of 0.05, betting results are 879.6178308650666
With a cutoff of 0.1, betting results are 793.4744849636562
With a cutoff of 0.15, betting results are 135.29673446554182
With a cutoff of 0.2, betting results are -200.01257697979014
With a cutoff of 0.25, betting results are -206.78506375227687
With a cutoff of 0.3, betting results are 75.0
With a cutoff of 0.35, betting results are 0.0
With a cutoff of 0.4, betting results are 0.0
With a cutoff of 0.45, betting results are 0.0
With a cutoff of 0.5, betting results are 0.0
For a 10 minimum, the model returns 832.7550857670275
For a 15 minimum, the model returns 863.900195399942
For a 20 minimum, the model returns 560.0138748353775
For a 25 minimum, the model returns -193.3439395735598


### LGBM

In [7]:
# Straight

def calculate_payoff_and_result(row):
    # Calculating Payoff
    if row.Predicted_Result_LGBM == 1:
        if row.Fighter_1_Odds>0:
            payoff = (row.Fighter_1_Odds/100)*row.Bet
        else:
            payoff = row.Bet/((abs(row.Fighter_1_Odds)/100))
    else:
        if row.Fighter_2_Odds>0:
            payoff = (row.Fighter_2_Odds/100)*row.Bet
        else:
            payoff = row.Bet/((abs(row.Fighter_2_Odds)/100))
    # Calculating Bet Result
    if row.Predicted_Result_LGBM == row.result_y:
        bet_result = payoff
    else:
        bet_result = -(row.Bet)
    
    return bet_result
            
profit_df['Bet'] = 100
profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result, axis = 1)

profit_df.Bet_Result.sum()

203.96138799985192

In [8]:
# Veteran fights only

for num_fights in [10, 15, 20, 25]:

    profit_df['Fights_1'] = profit_df.wins_1 + profit_df.losses_1
    profit_df['Fights_2'] = profit_df.wins_2 + profit_df.losses_2

    test = profit_df[(profit_df.Fights_1 > num_fights) | (profit_df.Fights_2 > num_fights)]
    results = test.Bet_Result.sum()
    
    print(f'For a {num_fights} minimum, the model returns {results}')

For a 10 minimum, the model returns 378.63710444027413
For a 15 minimum, the model returns 651.7158525171246
For a 20 minimum, the model returns 1178.2195238699337
For a 25 minimum, the model returns 541.4292511877229


In [13]:
# Betting with only a certain odds differential

def calculate_odds_internal(odds):
    if odds<0:
        return (abs(odds)/(abs(odds)+100))
    if odds>0:
        return (100/(odds+100))

def calculate_bets_internal(row, diff):
    bet = 0
    
    if row.Prediction_LGBM_Winner - calculate_odds_internal(row.Fighter_1_Odds) >= diff:
        bet = 100
    if (1.0 - row.Prediction_LGBM_Winner) - calculate_odds_internal(row.Fighter_2_Odds) >= diff:
        bet = 100
    
    return bet

def calculate_payoff_and_result_internal(row):

    if row.Bet > 0:
        # Calculating Payoff
        if row.Predicted_Result_LGBM == 1:
            if row.Fighter_1_Odds>0:
                payoff = (row.Fighter_1_Odds/100)*row.Bet
            else:
                payoff = row.Bet/((abs(row.Fighter_1_Odds)/100))
        else:
            if row.Fighter_2_Odds>0:
                payoff = (row.Fighter_2_Odds/100)*row.Bet
            else:
                payoff = row.Bet/((abs(row.Fighter_2_Odds)/100))
        # Calculating Bet Result
        if row.Predicted_Result_LGBM == row.result_y:
            bet_result = payoff
        else:
            bet_result = -(row.Bet)

    else:
        bet_result = 0

    return bet_result

best_diff = 0
best_profit = 0

for i in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
    
    profit_df['Bet'] = profit_df.apply(calculate_bets_internal, diff = i, axis = 1)
    profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result_internal, axis = 1)

    print(f'With a cutoff of {i}, betting results are {profit_df.Bet_Result.sum()}')
    
    if float(profit_df.Bet_Result.sum()) > best_profit:
        best_diff = i
    
    if profit_df.Bet_Result.sum() > best_profit:
        best_profit = profit_df.Bet_Result.sum()

# Veteran fights only

profit_df['Bet'] = profit_df.apply(calculate_bets_internal, diff = best_diff, axis = 1)
profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result_internal, axis = 1)

for num_fights in [10, 15, 20, 25]:

    profit_df['Fights_1'] = profit_df.wins_1 + profit_df.losses_1
    profit_df['Fights_2'] = profit_df.wins_2 + profit_df.losses_2

    test = profit_df[(profit_df.Fights_1 > num_fights) | (profit_df.Fights_2 > num_fights)]
    results = test.Bet_Result.sum()
    
    print(f'For a {num_fights} minimum, the model returns {results}')

With a cutoff of 0.05, betting results are 344.7361169109278
With a cutoff of 0.1, betting results are 480.45590200411436
With a cutoff of 0.15, betting results are 69.87274782125422
With a cutoff of 0.2, betting results are -260.2439335887612
With a cutoff of 0.25, betting results are -103.79948914431675
With a cutoff of 0.3, betting results are -200
With a cutoff of 0.35, betting results are -100
With a cutoff of 0.4, betting results are 0
With a cutoff of 0.45, betting results are 0
With a cutoff of 0.5, betting results are 0
For a 10 minimum, the model returns 480.4559020041143
For a 15 minimum, the model returns 567.5101025461198
For a 20 minimum, the model returns 1094.6529596889768
For a 25 minimum, the model returns 483.5916346561321


### RL

In [ ]:
# Setting up environment and data

# Loading SAC model
model = SAC.load('SAC_model.zip')

# Environment
class BettingEnv(gym.Env):
    
    def __init__(self, df, initial_funds):
        self.df = df
        self.current_step = 0
        self.initial_funds = initial_funds
        self.current_funds = initial_funds
        # Actions of the format Team 1 x%, Team 2 x%, No bet, etc.
        self.action_space = gym.spaces.Box(
          low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(17,), dtype=np.float32)
    
    def step(self, action):
        self.current_step += 1
        action_type = action[0]
        action_amount = (action[1] * self.current_funds) / 100.0
        
        if self.current_step >= len(self.df):
            done = True
            reward = 0
            obs = self.df.loc[len(self.df) - 1, ['reach_diff', 'age_diff', 'slpm_diff', 'sapm_diff', 'td_acc_diff', 'td_def_diff',
              'td_avg_diff', 'sub_avg_diff', 'strk_acc_diff', 'strk_def_diff', 'wins_diff',
              'losses_diff', 'win_pct_diff', 'weight_1', 'age_1', 'Prediction_1_lr', 'Prediction_rf']].values
            
        else:
            done = False
            row = self.df.loc[self.current_step, :]
            obs = row[['reach_diff', 'age_diff', 'slpm_diff', 'sapm_diff', 'td_acc_diff', 'td_def_diff',
              'td_avg_diff', 'sub_avg_diff', 'strk_acc_diff', 'strk_def_diff', 'wins_diff',
              'losses_diff', 'win_pct_diff', 'weight_1', 'age_1', 'Prediction_1_lr', 'Prediction_rf']].values
            
            if action_type < 1:
                # Betting on fighter 1
                if row[['Fighter_1_Odds']].values[0] > 0:
                    payoff = (row[['Fighter_1_Odds']].values[0]/100)*action_amount
                else:
                    payoff = action_amount/((abs(row[['Fighter_1_Odds']].values[0])/100))
                # Determining reward based on result
                if row[['result']].values[0] == 1:
                    reward = payoff
                else:
                    reward = -(action_amount)
                
            elif action_type < 2:
                # Determining payoff (away team)
                if row[['Fighter_2_Odds']].values[0] > 0:
                    payoff = (row[['Fighter_2_Odds']].values[0]/100)*action_amount
                else:
                    payoff = action_amount/((abs(row[['Fighter_2_Odds']].values[0])/100))
                # Determining reward based on result
                if row[['result']].values[0] == 0:
                    reward = payoff
                else:
                    reward = -(action_amount)
                
            else:
                # No bet
                reward = 0

        self.current_funds += reward
        return obs, reward, done, {}
    
    def reset(self):
        self.current_step = 0
        self.current_funds = self.initial_funds
        return self.df.loc[self.current_step, ['reach_diff', 'age_diff', 'slpm_diff', 'sapm_diff', 'td_acc_diff', 'td_def_diff',
              'td_avg_diff', 'sub_avg_diff', 'strk_acc_diff', 'strk_def_diff', 'wins_diff',
              'losses_diff', 'win_pct_diff', 'weight_1', 'age_1', 'Prediction_1_lr', 'Prediction_rf']]
    
    def render(self, mode='human'):
        # Render the environment
        print(f"Current funds: {self.current_funds}")

# Renaming columns to fit environment
merged.columns = ['fighter_1', 'weight_1', 'reach_1', 'age_1', 'slpm_1', 'sapm_1',
       'td_avg_1', 'sub_avg_1', 'strk_acc_1', 'strk_def_1', 'td_acc_1',
       'td_def_1', 'wins_1', 'losses_1', 'fighter_2', 'weight_2', 'reach_2',
       'age_2', 'slpm_2', 'sapm_2', 'td_avg_2', 'sub_avg_2', 'strk_acc_2',
       'strk_def_2', 'td_acc_2', 'td_def_2', 'wins_2', 'losses_2', 'result_x',
       'SUB_OVR_x', 'KO_OVR_x', 'reach_diff', 'age_diff', 'slpm_diff',
       'sapm_diff', 'td_acc_diff', 'td_def_diff', 'td_avg_diff',
       'sub_avg_diff', 'strk_acc_diff', 'strk_def_diff', 'wins_diff',
       'losses_diff', 'win_pct_1', 'win_pct_2', 'win_pct_diff',
       'Prediction_RF_Winner', 'Prediction_GB_Winner', 'Prediction_1_lr',
       'Prediction_RF_SUB', 'Prediction_GB_SUB', 'Prediction_LR_SUB',
       'Prediction_RF_KO', 'Prediction_GB_KO', 'Prediction_LR_KO', 'Date',
       'result_y', 'KO_OVR_y', 'SUB_OVR_y', 'Prediction_rf',
       'Predicted_Result_GB', 'Accurate_RF', 'Accurate_GB', 'Predicted_Sub_RF',
       'Predicted_Sub_GB', 'Accurate_RF_SUB', 'Accurate_GB_SUB',
       'Predicted_KO_RF', 'Predicted_KO_GB', 'Accurate_RF_KO',
       'Accurate_GB_KO']

In [36]:
%%capture

# Generating predictions

rl_prediction_columns = ['reach_diff', 'age_diff', 'slpm_diff', 'sapm_diff', 'td_acc_diff', 'td_def_diff',
              'td_avg_diff', 'sub_avg_diff', 'strk_acc_diff', 'strk_def_diff', 'wins_diff',
              'losses_diff', 'win_pct_diff', 'weight_1', 'age_1', 'Prediction_1_lr', 'Prediction_rf']
rl_data = merged[rl_prediction_columns]
for col in rl_prediction_columns:
    rl_data[col] = rl_data[col].astype('float')

scaler = StandardScaler()
rl_data[rl_prediction_columns] = scaler.fit_transform(rl_data[rl_prediction_columns])

rl_data['Prediction_RL'] = rl_data.apply(lambda x: model.predict(x), axis = 1)

rl_data['Predicted_Winner_RL'] = rl_data.Prediction_RL.apply(lambda x: 1 if x[0][0] < 1 else 0 if x[0][0] < 2 else -1)

# Concatenating predictions to above table
merged = pd.concat([merged, rl_data[['Predicted_Winner_RL']]], axis = 1)
merged['Accurate_RL'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Winner_RL else 0, axis = 1)

# Actual Profit

In [50]:
# Importing data
bets = pd.read_csv('mma_bets.csv', index_col = 0)
results = pd.read_csv('mma_data.csv', index_col = 0)[['fighter_1', 'fighter_2', 'result']]

# Appending results to bets data
bets['Result'] = -5
for index, row in bets.iterrows():
    
    # Applying fuzzy logic to get right fight
    results['FUZZ_1'] = results.fighter_1.apply(lambda x: fuzz.ratio(x, row.Fighter_1))
    results['FUZZ_2'] = results.fighter_1.apply(lambda x: fuzz.ratio(x, row.Fighter_2))
    results['FUZZ_3'] = results.fighter_2.apply(lambda x: fuzz.ratio(x, row.Fighter_1))
    results['FUZZ_4'] = results.fighter_2.apply(lambda x: fuzz.ratio(x, row.Fighter_2))
    relevant_row = results[((results.FUZZ_1 > 50) | (results.FUZZ_2 > 50)) & 
                          ((results.FUZZ_3 > 50) | (results.FUZZ_4 > 50))]
    try:
        result = relevant_row.result.values[0]
    except:
        continue
    
    # Seeing if there's a valid result for the fight
    if result < 0:
        continue
    
    # Appending result
    if relevant_row.FUZZ_1.values[0] > 50:
        pass
    else:
        if result == 1:
            result = 0
        else:
            result = 1
    bets.loc[index, 'Result'] = result

bets = bets[bets.Result != -5]

In [67]:
# Determining if bet hit - LGBM

bets_lgbm = bets[(bets.Bet_LGBM != 'No Bet') & (bets.Bet_LGBM != 'No bet')]

for index, row in bets_lgbm.iterrows():
    # Determining predicted winner
    predicted_winner = row.Bet_LGBM.split()[3:]
    if len(predicted_winner) == 2:
        predicted_winner = predicted_winner[0] +  ' ' + predicted_winner[1]
    else:
        predicted_winner = predicted_winner[0] +  ' ' + predicted_winner[1] + ' ' + predicted_winner[2]
    # Determining actual winner
    if row.Result == 1:
        winner = row.Fighter_1
    else:
        winner = row.Fighter_2

In [65]:
predicted_winner

['Cody', 'Stamann']

In [66]:
bets_lgbm

,Fighter_1,Fighter_2,Fighter_1_Odds,Fighter_2_Odds,Prediction_GB_Winner,Bet_GB,Bet_LGBM,Prediction_LGBM_Winner,Result
43,Cody Stamann,Douglas Silva De Andrade,-160.0,132.0,0.655476,No bet,Bet 100 on Cody Stamann,0.606628,0
45,Matt Brown,Court McGee,175.0,-215.0,0.547481,Bet 100 on Matt Brown,Bet 100 on Matt Brown,0.565986,1
46,Tim Means,Alex Morono,180.0,-220.0,0.621472,Bet 100 on Tim Means,Bet 100 on Tim Means,0.559486,0
47,Ihor Potieria,Carlos Ulberg,325.0,-425.0,0.397745,Bet 100 on Ihor Potieria,Bet 100 on Carlos Ulberg,0.384936,0
49,Anthony Smith,Johnny Walker,-110.0,-115.0,0.437782,No bet,Bet 100 on Johnny Walker,0.456676,0
52,Takashi Sato,Themba Gorimbo,100.0,-120.0,0.483182,No bet,Bet 100 on Themba Gorimbo,0.442653,0
53,Natalia Silva,Victoria Leonardo,-1000.0,625.0,0.705182,Bet 100 on Victoria Leonardo,Bet 100 on Natalia Silva,0.707150,0
54,Chase Hooper,Nick Fiore,110.0,-143.0,0.545326,Bet 100 on Chase Hooper,Bet 100 on Chase Hooper,0.569070,1
55,Ilir Latifi,Rodrigo Nascimento,155.0,-190.0,0.362157,No bet,Bet 100 on Rodrigo Nascimento,0.370576,0
58,Diego Ferreira,Michael Johnson,-160.0,132.0,0.642757,No bet,Bet 100 on Diego Ferreira,0.624411,1


In [73]:
test = pd.read_html('https://matchstat.com/tennis/player/Albert%20Ramos-Vinolas')

In [85]:
test[-4]

,year,sum.,hard,clay,i.hard,grass
0,2023,11/22,0/6,11/15,0/0,0/1
1,2022,27/28,4/7,21/15,2/4,0/2
2,2021,24/26,5/8,19/12,0/3,0/3
3,2020,9/14,2/3,7/9,0/2,0/0
4,2019,48/32,8/9,40/20,0/2,0/1
5,2018,22/30,4/12,15/14,3/3,0/1
6,2017,34/31,8/12,22/13,2/4,2/2
7,2016,34/31,9/12,19/13,4/4,2/2
8,2015,39/30,9/7,29/18,0/3,1/2
9,2014,61/31,3/4,56/24,2/3,0/0
